In [1]:
# Dependencies 
import pandas as pd
import numpy as np
import os

In [2]:
# load csv
file_path_school = os.path.join('schools_complete.csv')
file_path_student = os.path.join('students_complete.csv')

In [3]:
# read with pandas
df_school = pd.read_csv(file_path_school)
df_school.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [4]:
# read with pandas
df_student = pd.read_csv(file_path_student)
df_student.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [5]:
#########################
### District Summary ###
########################

# Calculate Total Schools
total_schools = len(df_school['School ID'])

# Calculate Total Students
total_students = len(df_student['Student ID'])

# Calculate Total Budget
total_budget = df_school['budget'].sum()

# Calculate Average Math Score
average_math = df_student['math_score'].mean()

# Calculate Average Reading Score
average_reading = df_student['reading_score'].mean()

# Calculate % Passing Math
passing_math_df = df_student.loc[df_student['math_score']>70]
passing_math = len(passing_math_df) / len(df_student['math_score']) * 100

# Calculate % Passing Reading
passing_reading_df = df_student.loc[df_student['reading_score']>70]
passing_reading = len(passing_reading_df) / len(df_student['reading_score']) * 100

# Calculate Overall Passing Rate (Average of the above two)
overall_passing_rate = (passing_math + passing_reading)/2

# Summary of district's key metrics
district_summary = pd.DataFrame({'Total Schools': total_schools,
                                'Total Students': total_students,
                                'Total Budget': total_budget,
                                'Average Math Score': average_math,
                                'Average Reading Score': average_reading,
                                '% Passing Math': passing_math,
                                '% Passing Reading': passing_reading,
                                'Overall Passing Rate': overall_passing_rate}, 
                               index=[0])

# format total students with comma
district_summary['Total Students'] = district_summary.apply(lambda x: "{:,}".format(x['Total Students']), axis=1)

# format to budget to currency
district_summary['Total Budget'] = district_summary['Total Budget'].map('${:,.2f}'.format)

# sort columns
district_summary_sorted = district_summary[['Total Schools','Total Students','Total Budget','Average Math Score',
                                           'Average Reading Score','% Passing Math','% Passing Reading','Overall Passing Rate']]

district_summary_sorted

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170.0","$24,649,428.00",78.985371,81.87784,72.392137,82.971662,77.681899


In [6]:
#######################
### School Summary ###
#######################

# Use columns from df_school and rename columns
school_data = df_school[['name', 'type','budget']]
school_data_renamed = school_data.rename(columns={'name':'School Name', 'type':'School Type','budget':'Total School Budget'})

# Calculate Per Student Budget and create data frame
student_budget_calc = df_school['budget']/df_school['size']
student_budget_df = pd.DataFrame({'Per Student Budget': student_budget_calc})

# merge data sets and change index to School Name
school_data_updated = pd.merge(school_data_renamed, student_budget_df, left_index=True, right_index=True)
school_data_index = school_data_updated.set_index('School Name')

# Count total students per school and add to school data
school_counts = df_student['school'].value_counts()

# Calculate average Math Score per school and add to school data
grouped_by_school = df_student.groupby('school')
avg_math_by_school = grouped_by_school['math_score'].mean()

# Calculate average Reading Score per schoola and add to school data
avg_read_by_school = grouped_by_school['reading_score'].mean()

# Calculate % passing math
count_mathscores_byschool = grouped_by_school['math_score'].count()
df_passing_math_only = passing_math_df[['school','math_score']]
group_passing_math_byschool = df_passing_math_only.groupby('school')
count_passing_math_byschool = group_passing_math_byschool['math_score'].count()
percent_passing_math_byschool = count_passing_math_byschool / count_mathscores_byschool * 100

# Calculate % passing reading
count_readscores_byschool = grouped_by_school['reading_score'].count()
df_passing_read_only = passing_reading_df[['school','reading_score']]
group_passing_read_byschool = df_passing_read_only.groupby('school')
count_passing_read_byschool = group_passing_read_byschool['reading_score'].count()
percent_passing_read_byschool = count_passing_read_byschool / count_readscores_byschool * 100

# Overall Passing Rate (Average of the above two)
overall_passing_rate_byschool = (percent_passing_math_byschool + percent_passing_read_byschool)/2

# create dataframe summarizing calculations and merge with school data
school_summary = pd.DataFrame({'Total Students': school_counts,
                               'Average Math Score': avg_math_by_school,
                               'Average Reading Score': avg_read_by_school,
                               '% Passing Math': percent_passing_math_byschool,
                               '% Passing Reading': percent_passing_read_byschool,
                               '% Overall Passing Rate': overall_passing_rate_byschool})
school_summary_merged = pd.merge(school_data_index, school_summary, left_index=True, right_index=True)

# format and sortcolumns
school_summary_sorted_column = school_summary_merged[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget',
                                               'Average Math Score','Average Reading Score','% Passing Math', 
                                               '% Passing Reading','% Overall Passing Rate']]

school_summary_sorted_column['Total Students'] = school_summary_sorted_column.apply(lambda x: "{:,}".format(x['Total Students']), axis=1)
school_summary_sorted_column['Total School Budget'] = school_summary_sorted_column['Total School Budget'].map('${:,.2f}'.format)
school_summary_sorted_column['Per Student Budget'] = school_summary_sorted_column['Per Student Budget'].map('${:,.2f}'.format)

school_summary_sorted_column

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.359455,83.725724,89.892107,92.617831,91.254969
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,91.553134
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,71.965434
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867


In [7]:
##############################
### Top Performing Schools ###
##############################

# sort school summary by overall passing rate
top_passing_rate_df = school_summary_sorted_column.sort_values('% Overall Passing Rate', ascending=False)

# display first 5 rows
top_performing = top_passing_rate_df[:5]
top_performing

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867
Wright High School,Charter,"1,800","$1,049,400.00",$583.00,83.682222,83.955000,90.277778,93.444444,91.861111
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183


In [8]:
#################################
### Bottom Performing Schools ### 
#################################

# sort school summary by overall passing rate
bottom_passing_rate_df = school_summary_sorted_column.sort_values('% Overall Passing Rate')

# display first 5 rows
bottom_performing = bottom_passing_rate_df[:5]
bottom_performing

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,64.066017,77.744436,70.905226
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098


In [9]:
############################
### Math Scores by Grade ###
############################

# group by school and grade
grouped_grade = df_student.groupby(['school','grade'])

# create data frame with averages
avg_math_bygrade = pd.DataFrame(grouped_grade['math_score'].mean())

# reshape data fame
math_by_grade = avg_math_bygrade.reset_index().pivot('school', 'grade', 'math_score')
math_by_grade_sortedcol = math_by_grade[['9th','10th','11th','12th']]
math_by_grade_sortedcol

grade,9th,10th,11th,12th
school,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [10]:
###############################
### Reading Scores by Grade ###
###############################

# create data frame with averages
avg_reading_bygrade = pd.DataFrame(grouped_grade['reading_score'].mean())

# reshape data fame
reading_by_grade = avg_reading_bygrade.reset_index().pivot('school', 'grade', 'reading_score')
reading_by_grade_sortedcol = reading_by_grade[['9th','10th','11th','12th']]
reading_by_grade_sortedcol

grade,9th,10th,11th,12th
school,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [11]:
#################################
### Scores by School Spending ###
#################################

# extract columns from school summary data frame
col_spend = [2,6,7,4,5,3]
by_spending = school_summary_merged.iloc[:,col_spend]

# create bins and add to data frame
bins_spending = [0, 600, 620, 640, 660]
group_names_spending = ["< $600", "$600-620","$620-640", "> $640"]
pd.cut(by_spending['Per Student Budget'], bins_spending, labels=group_names_spending)
by_spending['Spending Ranges (Per Student)'] = pd.cut(by_spending['Per Student Budget'], bins_spending, labels=group_names_spending)

# Creating a group based off of the bins
by_spending_grouped = by_spending.groupby('Spending Ranges (Per Student)')
grouped_spending_df = by_spending_grouped.max()
col_spend2 = [1,2,3,4,5]
grouped_spending_summary =  grouped_spending_df.iloc[:,col_spend2]
grouped_spending_summary

C:\Users\Katrina\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
< $600,83.803279,83.989488,90.932983,93.864370,92.093736
$600-620,83.839917,84.044699,91.683992,92.203742,91.943867
$620-640,83.418349,83.848930,90.214067,93.392371,91.559633
> $640,77.289752,81.182722,65.753925,78.813850,71.631982


In [12]:
#############################
### Scores by School Size ###
#############################

# extract columns from school summary data frame
col_size = [8,6,7,3,4,5]
by_size = school_summary_merged.iloc[:,col_size]

# create bins
bins_size = [0, 2000, 3500, 5000]
group_names_size = ['Small (<2000)', 'Medium (2000-3500)', 'Large (>3500)']
pd.cut(by_size['Total Students'], bins_size, labels=group_names_size)
by_size['School Size'] = pd.cut(by_size['Total Students'], bins_size, labels=group_names_size)

# Creating a group based off of the bins
by_size_grouped = by_size.groupby('School Size')
grouped_size_df = by_size_grouped.max()
col_size2 = [1,2,3,4,5]
grouped_size_summary =  grouped_size_df.iloc[:,col_size2]
grouped_size_summary

C:\Users\Katrina\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Average Math Score,Average Reading Score,% Overall Passing Rate,% Passing Math,% Passing Reading
School Size,,,,,
Small (<2000),83.839917,84.044699,91.943867,91.683992,93.864370
Medium (2000-3500),83.274201,83.989488,92.093736,90.932983,93.254490
Large (>3500),77.289752,81.033963,71.965434,64.746494,79.300643


In [13]:
#############################
### Scores by School Type ###
#############################

# extract columns from school summary data frame
col_type = [0,6,7,4,5,3]
by_type = school_summary_merged.iloc[:,col_type]

grouped_type = by_type.groupby('School Type')
grouped_type.max().head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.839917,84.044699,91.683992,93.864370,92.093736
District,77.289752,81.182722,65.753925,79.300643,71.965434
